In [ ]:
TRAIN="http://www.i3s.unice.fr/~riveill/dataset/MIMIC-III-readmission/train.csv.zip"
VAL="http://www.i3s.unice.fr/~riveill/dataset/MIMIC-III-readmission/val.csv.zip"
TEST="http://www.i3s.unice.fr/~riveill/dataset/MIMIC-III-readmission/test.csv.zip"

In [ ]:
import ssl
import pandas as pd
import matplotlib.pyplot as plt
ssl._create_default_https_context = ssl._create_unverified_context
train = pd.read_csv(TRAIN)
test = pd.read_csv(TEST)

In [ ]:
to_drop = ['SUBJECT_ID', 'HADM_ID'] 
train = train.drop(to_drop, axis=1)
test = test.drop(to_drop, axis=1)

In [ ]:
to_drop = ['DAYS_NEXT_ADMIT', 'NEXT_ADMITTIME', 'MARITAL_STATUS', 'DIAGNOSIS']
train = train.drop(to_drop, axis=1)
test = test.drop(to_drop, axis=1)
train.head()

,ADMITTIME,DISCHTIME,ADMISSION_TYPE,DEATHTIME,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,TEXT,GENDER,DOB,blood,circulatory,congenital,digestive,endocrine,genitourinary,infectious,injury,mental,misc,muscular,neoplasms,nervous,pregnancy,prenatal,respiratory,skin,OUTPUT_LABEL
0,2163-01-20 18:39:00,2163-01-24 08:00:00,EMERGENCY,2163-01-26 08:00:00,DEAD/EXPIRED,Medicare,OTHER/UNKNOWN,Admission Date: [**2163-1-20**] ...,M,2087-09-24 00:00:00,0,4,0,0,1,0,0,2,0,0,0,0,1,0,0,0,0,1
1,2107-01-23 02:45:00,2107-01-26 14:00:00,EMERGENCY,NaN,HOME HEALTH CARE,Medicare,WHITE,Admission Date: [**2107-1-23**] Discharge...,M,2028-04-07 00:00:00,0,2,0,2,1,0,0,1,2,0,0,0,0,0,0,1,0,0
2,2134-06-24 23:30:00,2134-07-02 17:45:00,EMERGENCY,NaN,REHAB/DISTINCT PART HOSP,Medicaid,WHITE,Admission Date: [**2134-6-24**] ...,M,2087-11-05 00:00:00,1,2,0,0,6,1,1,4,1,2,1,0,3,0,0,4,0,0
3,2202-11-02 12:32:00,2202-11-05 14:20:00,EMERGENCY,NaN,HOME,Medicare,BLACK/AFRICAN AMERICAN,Admission Date: [**2202-11-2**] Discharge...,F,2168-10-06 00:00:00,0,1,0,1,3,1,0,1,0,0,0,0,0,0,0,1,1,1
4,2149-05-13 12:23:00,2149-05-26 20:00:00,EMERGENCY,NaN,HOME HEALTH CARE,Medicare,OTHER/UNKNOWN,Admission Date: [**2149-5-13**] ...,F,2091-10-09 00:00:00,0,8,0,0,3,3,0,3,0,0,0,0,0,0,0,2,1,0


In [ ]:
train = train.loc[train['DEATHTIME'].isnull()].drop(['DEATHTIME'], axis=1)
test = test.loc[test['DEATHTIME'].isnull()].drop(['DEATHTIME'], axis=1)
train.head()

,ADMITTIME,DISCHTIME,ADMISSION_TYPE,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,TEXT,GENDER,DOB,blood,circulatory,congenital,digestive,endocrine,genitourinary,infectious,injury,mental,misc,muscular,neoplasms,nervous,pregnancy,prenatal,respiratory,skin,OUTPUT_LABEL
1,2107-01-23 02:45:00,2107-01-26 14:00:00,EMERGENCY,HOME HEALTH CARE,Medicare,WHITE,Admission Date: [**2107-1-23**] Discharge...,M,2028-04-07 00:00:00,0,2,0,2,1,0,0,1,2,0,0,0,0,0,0,1,0,0
2,2134-06-24 23:30:00,2134-07-02 17:45:00,EMERGENCY,REHAB/DISTINCT PART HOSP,Medicaid,WHITE,Admission Date: [**2134-6-24**] ...,M,2087-11-05 00:00:00,1,2,0,0,6,1,1,4,1,2,1,0,3,0,0,4,0,0
3,2202-11-02 12:32:00,2202-11-05 14:20:00,EMERGENCY,HOME,Medicare,BLACK/AFRICAN AMERICAN,Admission Date: [**2202-11-2**] Discharge...,F,2168-10-06 00:00:00,0,1,0,1,3,1,0,1,0,0,0,0,0,0,0,1,1,1
4,2149-05-13 12:23:00,2149-05-26 20:00:00,EMERGENCY,HOME HEALTH CARE,Medicare,OTHER/UNKNOWN,Admission Date: [**2149-5-13**] ...,F,2091-10-09 00:00:00,0,8,0,0,3,3,0,3,0,0,0,0,0,0,0,2,1,0
5,2156-06-02 08:00:00,2156-06-08 13:30:00,ELECTIVE,HOME HEALTH CARE,Medicare,WHITE,"Name: [**Known lastname 10188**], [**Known fi...",M,2070-10-02 00:00:00,0,3,0,1,2,0,0,1,1,0,0,0,0,0,0,0,0,0


In [ ]:
output = ['OUTPUT_LABEL']
remove = ['ADMITTIME', 'DISCHTIME', # It's possible to use length of stay: 'DISCHTIME'-'ADMITTIME'
          'TEXT',
          'DOB', # It's possible to use the age at admission time: 'ADMITTIME'-'DOB'
         ]

features = list(train.columns)
for x in output+remove:
    features.remove(x)

In [ ]:
X_train = train[features]
y_train = train[output]

X_test = test[features]
y_test = test[output]

X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)
X_train.head()

,blood,circulatory,congenital,digestive,endocrine,genitourinary,infectious,injury,mental,misc,muscular,neoplasms,nervous,pregnancy,prenatal,respiratory,skin,ADMISSION_TYPE_ELECTIVE,ADMISSION_TYPE_EMERGENCY,ADMISSION_TYPE_URGENT,DISCHARGE_LOCATION_DISC-TRAN CANCER/CHLDRN H,DISCHARGE_LOCATION_DISCH-TRAN TO PSYCH HOSP,DISCHARGE_LOCATION_HOME,DISCHARGE_LOCATION_HOME HEALTH CARE,DISCHARGE_LOCATION_HOME WITH HOME IV PROVIDR,DISCHARGE_LOCATION_HOSPICE-HOME,DISCHARGE_LOCATION_HOSPICE-MEDICAL FACILITY,DISCHARGE_LOCATION_ICF,DISCHARGE_LOCATION_LEFT AGAINST MEDICAL ADVI,DISCHARGE_LOCATION_LONG TERM CARE HOSPITAL,DISCHARGE_LOCATION_OTHER FACILITY,DISCHARGE_LOCATION_REHAB/DISTINCT PART HOSP,DISCHARGE_LOCATION_SHORT TERM HOSPITAL,DISCHARGE_LOCATION_SNF,INSURANCE_Government,INSURANCE_Medicaid,INSURANCE_Medicare,INSURANCE_Private,INSURANCE_Self Pay,ETHNICITY_ASIAN,ETHNICITY_BLACK/AFRICAN AMERICAN,ETHNICITY_HISPANIC/LATINO,ETHNICITY_OTHER/UNKNOWN,ETHNICITY_WHITE,GENDER_F,GENDER_M
1,0,2,0,2,1,0,0,1,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1
2,1,2,0,0,6,1,1,4,1,2,1,0,3,0,0,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1
3,0,1,0,1,3,1,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
4,0,8,0,0,3,3,0,3,0,0,0,0,0,0,0,2,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
5,0,3,0,1,2,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1


In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import numpy as np
y_train = np.array(y_train).reshape(-1,)
y_test = np.array(y_test).reshape(-1,)
X_train.shape, y_train.shape, y_test.shape

((1842, 46), (1842,), (843,))

In [ ]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred = svclassifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[281 113]
 [176 273]]
              precision    recall  f1-score   support

           0       0.61      0.71      0.66       394
           1       0.71      0.61      0.65       449

    accuracy                           0.66       843
   macro avg       0.66      0.66      0.66       843
weighted avg       0.66      0.66      0.66       843



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

paramGrid = [
{'C':[0.1, 0.2, 0.5, 1, 2, 5, 10], 'kernel':['linear']},
{'C':[0.5, 1, 5, 10],'degree':[2,3],'coef0':[-1,0,1],'kernel':['poly']},
{'C':[0.1, 0.2, 0.5, 1, 2, 5, 10], 'kernel':['sigmoid']}
]
searchBestParams = GridSearchCV(svm.SVC(), paramGrid, scoring='precision_macro')
searchBestParams.fit(X_train, y_train)
#searchBestParams
means = searchBestParams.cv_results_['mean_test_score']
stds = searchBestParams.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, searchBestParams.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))


0.676 (+/-0.046) for {'C': 0.1, 'kernel': 'linear'}
0.675 (+/-0.043) for {'C': 0.2, 'kernel': 'linear'}
0.674 (+/-0.044) for {'C': 0.5, 'kernel': 'linear'}
0.673 (+/-0.042) for {'C': 1, 'kernel': 'linear'}
0.671 (+/-0.040) for {'C': 2, 'kernel': 'linear'}
0.672 (+/-0.038) for {'C': 5, 'kernel': 'linear'}
0.672 (+/-0.039) for {'C': 10, 'kernel': 'linear'}
0.339 (+/-0.020) for {'C': 0.5, 'coef0': -1, 'degree': 2, 'kernel': 'poly'}
0.599 (+/-0.048) for {'C': 0.5, 'coef0': -1, 'degree': 3, 'kernel': 'poly'}
0.660 (+/-0.084) for {'C': 0.5, 'coef0': 0, 'degree': 2, 'kernel': 'poly'}
0.689 (+/-0.053) for {'C': 0.5, 'coef0': 0, 'degree': 3, 'kernel': 'poly'}
0.677 (+/-0.040) for {'C': 0.5, 'coef0': 1, 'degree': 2, 'kernel': 'poly'}
0.667 (+/-0.036) for {'C': 0.5, 'coef0': 1, 'degree': 3, 'kernel': 'poly'}
0.340 (+/-0.022) for {'C': 1, 'coef0': -1, 'degree': 2, 'kernel': 'poly'}
0.600 (+/-0.050) for {'C': 1, 'coef0': -1, 'degree': 3, 'kernel': 'poly'}
0.649 (+/-0.041) for {'C': 1, 'coef0': 0, '